# Polarization Calculation 2000

In [2]:
import pandas as pd
import os
import time
import numpy as np

In [3]:
os.chdir(r"A:\UW-Madison\GIS SPRING 2023\Geog 575\Final_Project\SPSS_Processed\UKDA-5151-spss\spss\spss19")

In [4]:
df_residence = pd.read_csv(r"A:\UW-Madison\GIS SPRING 2023\Geog 575\Final_Project\SPSS_Processed\UKDA-5151-spss\spss\spss19\bhps_w10\jindresp.csv")

C:\Users\ramak\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,7,29,879,880,993,1104,1156,1212,1216,1217) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
print(list(df_residence.columns))

['Unnamed: 0', 'jhid', 'jpno', 'jdoid', 'jdoim', 'jdoiy4', 'jivsoih', 'jivsoim', 'jivlyr', 'jivstat2', 'jlknbrd', 'jlkmove', 'jlkmovy', 'jxpmove', 'jplnew', 'jplnowm', 'jplnowy4', 'jmovjb', 'jmovjba', 'jmovjbb', 'jmovjbc', 'jmovjbd', 'jmovjbe', 'jmovjbf', 'jmovjbg', 'jmovjbh', 'jmovjbi', 'jmovy1', 'jmovy2', 'jdoby', 'jsex', 'jjbstat', 'jedlyr', 'jedtype1', 'jedblyr1', 'jedbgm1', 'jedbgy1', 'jedenm1', 'jedeny1', 'jedenne1', 'jedfeea1', 'jedfeeb1', 'jedfeec1', 'jedfeed1', 'jedfeee1', 'jedfeef1', 'jedfeeg1', 'jedqual1', 'jedqnn1', 'jedqla1', 'jedqlb1', 'jedqlc1', 'jedqld1', 'jedqle1', 'jedqlf1', 'jedqlg1', 'jedqlh1', 'jedqli1', 'jedqlj1', 'jedqlan1', 'jedqlbn1', 'jedqlcn1', 'jedqldn1', 'jedqlen1', 'jedqlfn1', 'jedqlgn1', 'jedqlhn1', 'jedqlin1', 'jedqljn1', 'jedoqln1', 'jedoql1', 'jedmore1', 'jedtype2', 'jedblyr2', 'jedbgm2', 'jedbgy2', 'jedenm2', 'jedeny2', 'jedenne2', 'jedfeea2', 'jedfeeb2', 'jedfeec2', 'jedfeed2', 'jedfeee2', 'jedfeef2', 'jedfeeg2', 'jedqual2', 'jedqnn2', 'jedqla2', 'je

In [6]:
# Loading variables of interest from 1995 BHPS dataset into dataframe
df_residence = df_residence[["jopsocc","jopsocd","jopsoce","jfiyr","jfetype","jage12","jjbstatt","jregion2", "jplbornc_cc"]]

In [8]:
# Dropping rows if they donot have any spatial information
df_residence = df_residence[~df_residence["jregion2"].isin(["don't know", "inapplicable", "missing or wild", "refused"])]

In [19]:
# To view columns
df_residence.head()

,jopsocc,jopsocd,jopsoce,jfiyr,jfetype,jage12,jjbstatt,jregion2,jplbornc_cc,jfetype_encoded,jjbstatt_encoded,jopsocc_encoded
0,Agree,Disagree,Disagree,30849.408203,inapplicable,66.0,Self employed,London,inapplicable,-8,1,2
1,Agree,Neither agree/disagree,Disagree,2553.462891,inapplicable,63.0,Family care,London,inapplicable,-8,6,2
2,proxy and/or phone,proxy and/or phone,proxy and/or phone,0.000000,proxy and/or phone,45.0,proxy and/or phone,London,proxy and/or phone,-7,proxy and/or phone,-7
3,proxy and/or phone,proxy and/or phone,proxy and/or phone,0.000000,proxy and/or phone,19.0,proxy and/or phone,London,proxy and/or phone,-7,proxy and/or phone,-7
4,Agree,Agree,Disagree,6754.799316,inapplicable,21.0,"FT studt, school",London,inapplicable,-8,7,2


In [21]:
df_residence['jopsocc'].value_counts()

Neither agree/disagree    5819
Disagree                  4014
Agree                     3189
don't know                 873
Strongly disagree          579
proxy and/or phone         510
Strongly agree             429
Name: jopsocc, dtype: int64

In [37]:
# create an empty dataframe
output_df = pd.DataFrame()

# Group the dataframe by county and calculate standard deviation of age : "jage12"
output_df["std_dev_age"] = df_residence.groupby("jregion2")["jage12"].agg("std")

# Group the dataframe by county and number of people born in UK : "jplbornc_cc"
output_df["born"] = df_residence.groupby("jregion2")["jplbornc_cc"]

# Assuming inapplicable as people born in UK to all others
def native_share(x):
    pop_count = list(x["born"][1])
    native_count = pop_count.count("inapplicable")/len(pop_count)
    return native_count

# calculating fraction of natives in each county
output_df["native_share"] = output_df.apply(lambda x: native_share(x), axis = 1)

# Dropping "born" groupby column
output_df = output_df.drop(columns = ["born"], axis = 1)

# Creating afetype_dict based on PDF from 2000 BHPS data
jfetype_dict = {"Nursing school etc" : 1, "College of f educ" : 2, "Other trng establmnt" : 3, "Polytechnic" : 4, \
                "University" : 5, "None of the above" : 7, "don't know" : -1, "missing or wild" : -9, \
                    "inapplicable" : -8, "proxy and/or phone" : -7, "refused" : -2}


# Create a new column "jfetype_encoded" in the dataframe by replacing the values with encoded values from the dictionary
#df_residence["jfetype_encoded"] = df_residence["jfetype"].replace(jfetype_dict)
df_residence.loc[:, "jfetype_encoded"] = df_residence["jfetype"].replace(jfetype_dict)

# Group the dataframe by county and "jfetype_encoded"
output_df["education"] = df_residence.groupby("jregion2")["jfetype_encoded"]

# function to calculate dissimilarity in education
def dissimilarity_education(x):
    # loads tuple element into job
    job = x["education"][1]
    # loads frequency of each education type in county as list
    freq = list(job.value_counts())
    # calculates total number of groups
    s = len(freq)
    dissim_index = 0
    for i in freq:
        # formula of dissimilarity index
        dissim_index += 0.5 * abs(i-(1/s))
    return dissim_index

def region_name(x):
    return x["education"][0]

output_df["education_variability"] = output_df.apply(lambda x: dissimilarity_education(x), axis = 1)
output_df["region_name"] = output_df.apply(lambda x: region_name(x), axis = 1)

# Dropping "education" groupby column
output_df = output_df.drop(columns = ["education"], axis = 1)

# Creating ajbstatt_dict based on PDF from 2000 BHPS data
jjbstatt_dict = {"Self employed" : 1, "In paid employ" : 2, "Unemployed" : 3, "Retired" : 4, "Maternity leave" : 5, \
                    "Family care" : 6,  "FT studt, school" : 7, " LT sick, disabld" : 8, \
                    "Govt trng scheme" : 9, "Something else" : 10, "don't know" : -1, "missing or wild" : -9, \
                    "inapplicable" : -8, "proxy" : -7, "refused" : -2}


# Create a new column "jjbstatt_encoded" in the dataframe by replacing the values with encoded values from the dictionary
df_residence.loc[:, "jjbstatt_encoded"] = df_residence["jjbstatt"].replace(jjbstatt_dict)

# Group the dataframe by county and calculate the mean and standard deviation of the encoded values
output_df["job"] = df_residence.groupby("jregion2")["jjbstatt_encoded"]

# function to calculate dissimilarity in job
def dissimilarity_job(x):
    job = x["job"][1]
    freq = list(job.value_counts())
    s = len(freq)
    dissim_index = 0
    for i in freq:
        dissim_index += 0.5 * abs(i-(1/s))
    return dissim_index

# function to calculate fraction of employed people within each county
def frac_employed(x):
    job = list(x["job"][1])
    frac_emp = (job.count(2)/len(job))
    return frac_emp

output_df["job_variability"] = output_df.apply(lambda x: dissimilarity_job(x), axis = 1)
output_df ["frac_employed"] = output_df.apply(lambda x: frac_employed(x), axis = 1)

# Dropping "education" groupby column
output_df = output_df.drop(columns = ["job"], axis = 1)

# replacing all Not-A-Number cases to zero i.e., proxy values
#df_residence["jfiyr"] = df_residence["jfiyr"].replace("proxy and/or phone",0)
df_residence.loc[:, "jfiyr"] = df_residence["jfiyr"].replace("proxy and/or phone",0).astype("float32")

# Group the dataframe by county and calculate median based on self-reported annual income: "jfiyr"
output_df["median_income"] = df_residence.groupby("jregion2")["jfiyr"].agg(["median"])

# convert "afiyr" column to float data type
#df_residence["jfiyr"] = df_residence["jfiyr"].astype("float32")

# group incomes by county
output_df["income"] = df_residence.groupby("jregion2")["jfiyr"]

def gini_index(x):
    data = list(x["income"][1])
    # sort the data in ascending order
    sorted_data = np.sort(data)
    # calculate the cumulative sum of the sorted data
    cumsum_data = np.cumsum(sorted_data)
    # calculate the Lorenz curve values
    lorenz_curve = cumsum_data / cumsum_data[-1]
    # calculate the area under the Lorenz curve
    area_lorenz_curve = np.trapz(lorenz_curve, dx=1/len(data))
    # calculate the Gini index
    gini_index = 1 - 2 * area_lorenz_curve
    return gini_index

output_df["gini_index"] = output_df.apply(lambda x : gini_index(x), axis = 1)

# Drop "income" groupby column
output_df = output_df.drop(columns = ["income"], axis = 1)

output_df

C:\Users\ramak\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,std_dev_age,native_share,education_variability,region_name,job_variability,frac_employed,median_income,gini_index
jregion2,,,,,,,,
East Midlands,18.275323,0.958919,462.0,East Midlands,462.0,0.511351,8678.310547,0.449646
East of England,18.773603,0.959739,459.0,East of England,459.0,0.503808,9591.965820,0.493658
London,18.378665,0.923409,463.0,London,463.0,0.514563,10688.221680,0.505179
North East,18.756243,0.974206,251.5,North East,251.5,0.466270,8281.648438,0.456207
North West,18.888780,0.972397,633.5,North West,633.5,0.495268,9418.572266,0.455936
Scotland,18.180234,0.969923,1745.0,Scotland,1745.0,0.486107,8709.897461,0.453888
South East,18.342758,0.952381,734.5,South East,734.5,0.521769,10013.899414,0.463005
South West,19.189720,0.958635,483.0,South West,483.0,0.490176,8671.809570,0.472033
Wales,18.570539,0.959347,1500.0,Wales,1500.0,0.410863,8300.000000,0.468485


In [38]:
# Define the dictionary to encode the "jopsocc" column which is Statement 1 (S1)
# S1: "Private enterprise is the best way to solve Britain’s economic problems."
jopsocc_dict = {"Strongly agree": 1,"Agree":2, "Not agree, disagree": 3 , "Disagree":4 , "Strongly disagree": 5,
                "don't know": -1 , "missing or wild": -9, "inapplicable":-8,
                "proxy and/or phone":-7, "refused":-2, "Neither agree/disagree":0}

# Create a new column "jopsocc_encoded" in the dataframe by replacing the values with encoded values from the dictionary
#df_residence["jopsocc_encoded"] = df_residence["jopsocc"].replace(jopsocc_dict)
df_residence.loc[:, "jopsocc_encoded"] = df_residence["jopsocc"].replace(jopsocc_dict)

# Political polarization measurement by Lindqvist and Ostling (2010)
output_df["Lindqvist_Ostling_S1"] = df_residence.groupby("jregion2")["jopsocc_encoded"].agg(["var"])

# Create new columns based on absolute sum of encoded values of opinions and counts
output_df["abs_sum"] = df_residence.groupby("jregion2")["jopsocc_encoded"].apply(lambda x: x.abs().sum())
output_df["counts"] = df_residence.groupby("jregion2")["jopsocc_encoded"].apply(lambda x: x.count())

# Political polarization measurement by Abramowitz and Saunders (2008)
output_df["Abramowitz_Saunders_S1"] = output_df.apply(lambda x: x["abs_sum"]/x["counts"], axis = 1)

output_df = output_df.drop(columns = ["abs_sum", "counts"], axis = 1)

# Group the dataframe by county and jopsocc_encoded values
output_df["dkp"] = df_residence.groupby("jregion2")["jopsocc_encoded"]

# Political polarization measurement by Duca and Saving (2016)
def duca_saving(x):
    county_tuple = x["dkp"]
    freq = list(county_tuple[1].value_counts())
    val = list(county_tuple[1].unique())
    norm_freq = [i/sum(freq) for i in freq]
    outer_sum = 0
    for i in range(len(norm_freq)):
        nkit = norm_freq[i]
        inner_sum = 0
        for j in range(len(norm_freq)):
            npit = norm_freq[j]
            dkp = abs(val[i] - val[j])
            inner_sum += npit*dkp
    outer_sum += (nkit**2)*inner_sum
    
    return outer_sum

output_df["Duca_Saving_S1"] = output_df.apply(lambda x: duca_saving(x), axis = 1)

# Drop "income" groupby column
output_df = output_df.drop(columns = ["dkp"], axis = 1)

output_df

C:\Users\ramak\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,std_dev_age,native_share,education_variability,region_name,job_variability,frac_employed,median_income,gini_index,Lindqvist_Ostling_S1,Abramowitz_Saunders_S1,Duca_Saving_S1
jregion2,,,,,,,,,,,
East Midlands,18.275323,0.958919,462.0,East Midlands,462.0,0.511351,8678.310547,0.449646,5.897372,1.818378,0.001149
East of England,18.773603,0.959739,459.0,East of England,459.0,0.503808,9591.965820,0.493658,5.312424,1.746464,0.001776
London,18.378665,0.923409,463.0,London,463.0,0.514563,10688.221680,0.505179,7.371926,2.165049,0.003747
North East,18.756243,0.974206,251.5,North East,251.5,0.466270,8281.648438,0.456207,5.246600,2.007937,0.000443
North West,18.888780,0.972397,633.5,North West,633.5,0.495268,9418.572266,0.455936,5.196530,1.832808,0.005411
Scotland,18.180234,0.969923,1745.0,Scotland,1745.0,0.486107,8709.897461,0.453888,5.418960,2.101690,0.001089
South East,18.342758,0.952381,734.5,South East,734.5,0.521769,10013.899414,0.463005,5.743897,1.767347,0.000971
South West,19.189720,0.958635,483.0,South West,483.0,0.490176,8671.809570,0.472033,5.408182,1.805584,0.003612
Wales,18.570539,0.959347,1500.0,Wales,1500.0,0.410863,8300.000000,0.468485,6.236510,2.132956,0.003785


In [31]:
# Define the dictionary to encode the "jopsocd" column which is Statement 2 (S2)
# S2: "Major public services and industries ought to be in state ownership."

jopsocd_dict = {"Strongly agree": 1,"Agree":2, "Not agree, disagree": 3 , "Disagree":4 , "Strongly disagree": 5,
                "don't know": -1 , "missing or wild": -9, "inapplicable":-8, "proxy and/or phone":-7, "refused":-2, "Neither agree/disagree":0}

# Create a new column "jopsocc_encoded" in the dataframe by replacing the values with encoded values from the dictionary
#df_residence["jopsocd_encoded"] = df_residence["jopsocd"].replace(jopsocd_dict)
df_residence.loc[:, "jopsocd_encoded"] = df_residence["jopsocd"].replace(jopsocd_dict)

# Political polarization measurement by Lindqvist and Ostling (2010)
output_df["Lindqvist_Ostling_S2"] = df_residence.groupby("jregion2")["jopsocd_encoded"].agg(["var"])

# Create new columns based on absolute sum of encoded values of opinions and counts
output_df["abs_sum"] = df_residence.groupby("jregion2")["jopsocd_encoded"].apply(lambda x: x.abs().sum())
output_df["counts"] = df_residence.groupby("jregion2")["jopsocd_encoded"].apply(lambda x: x.count())

# Political polarization measurement by Abramowitz and Saunders (2008)
output_df["Abramowitz_Saunders_S2"] = output_df.apply(lambda x: x["abs_sum"]/x["counts"], axis = 1)

output_df = output_df.drop(columns = ["abs_sum", "counts"], axis = 1)

# Group the dataframe by county and jopsocc_encoded values
output_df["dkp"] = df_residence.groupby("jregion2")["jopsocd_encoded"]

# Political polarization measurement by Duca and Saving (2016)
def duca_saving(x):
    county_tuple = x["dkp"]
    freq = list(county_tuple[1].value_counts())
    val = list(county_tuple[1].unique())
    norm_freq = [i/sum(freq) for i in freq]
    outer_sum = 0
    for i in range(len(norm_freq)):
        nkit = norm_freq[i]
        inner_sum = 0
        for j in range(len(norm_freq)):
            npit = norm_freq[j]
            dkp = abs(val[i] - val[j])
            inner_sum += npit*dkp
    outer_sum += (nkit**2)*inner_sum
    
    return outer_sum

output_df["Duca_Saving_S2"] = output_df.apply(lambda x: duca_saving(x), axis = 1)

# Drop "income" groupby column
output_df = output_df.drop(columns = ["dkp"], axis = 1)

C:\Users\ramak\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [33]:
# Define the dictionary to encode the "jopsocd" column which is Statement 3 (S3)
# S3: "It is the government’s responsibility to provide a job for everyone who wants one."

jopsoce_dict = {"Strongly agree": 1,"Agree":2, "Not agree, disagree": 3 , "Disagree":4 , "Strongly disagree": 5,
                "don't know": -1 , "missing or wild": -9, "inapplicable":-8, "proxy and/or phone":-7, "refused":-2, "Neither agree/disagree":0}

# Create a new column "jopsocd_encoded" in the dataframe by replacing the values with encoded values from the dictionary
#df_residence["jopsoce_encoded"] = df_residence["jopsoce"].replace(jopsoce_dict)
df_residence.loc[:, "jopsoce_encoded"] = df_residence["jopsoce"].replace(jopsoce_dict)

# Political polarization measurement by Lindqvist and Ostling (2010)
output_df["Lindqvist_Ostling_S3"] = df_residence.groupby("jregion2")["jopsoce_encoded"].agg(["var"])

# Create new columns based on absolute sum of encoded values of opinions and counts
output_df["abs_sum"] = df_residence.groupby("jregion2")["jopsoce_encoded"].apply(lambda x: x.abs().sum())
output_df["counts"] = df_residence.groupby("jregion2")["jopsoce_encoded"].apply(lambda x: x.count())

# Political polarization measurement by Abramowitz and Saunders (2008)
output_df["Abramowitz_Saunders_S3"] = output_df.apply(lambda x: x["abs_sum"]/x["counts"], axis = 1)

output_df = output_df.drop(columns = ["abs_sum", "counts"], axis = 1)

# Group the dataframe by county and eopsocc_encoded values
output_df["dkp"] = df_residence.groupby("jregion2")["jopsoce_encoded"]

# Political polarization measurement by Duca and Saving (2016)
def duca_saving(x):
    county_tuple = x["dkp"]
    freq = list(county_tuple[1].value_counts())
    val = list(county_tuple[1].unique())
    norm_freq = [i/sum(freq) for i in freq]
    outer_sum = 0
    for i in range(len(norm_freq)):
        nkit = norm_freq[i]
        inner_sum = 0
        for j in range(len(norm_freq)):
            npit = norm_freq[j]
            dkp = abs(val[i] - val[j])
            inner_sum += npit*dkp
    outer_sum += (nkit**2)*inner_sum
    
    return outer_sum

output_df["Duca_Saving_S3"] = output_df.apply(lambda x: duca_saving(x), axis = 1)

# Drop "income" groupby column
output_df = output_df.drop(columns = ["dkp"], axis = 1)

output_df

C:\Users\ramak\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,std_dev_age,native_share,education_variability,region_name,job_variability,frac_employed,median_income,gini_index,Lindqvist_Ostling_S1,Abramowitz_Saunders_S1,Duca_Saving_S1,Lindqvist_Ostling_S2,Abramowitz_Saunders_S2,Duca_Saving_S2,Lindqvist_Ostling_S3,Abramowitz_Saunders_S3,Duca_Saving_S3
jregion2,,,,,,,,,,,,,,,,,
East Midlands,18.275323,0.958919,462.0,East Midlands,462.0,0.511351,8678.310547,0.449646,5.897372,1.818378,0.001149,6.021287,2.073514,0.004556,5.426529,2.431351,0.000332
East of England,18.773603,0.959739,459.0,East of England,459.0,0.503808,9591.965820,0.493658,5.312424,1.746464,0.001776,5.837405,2.103373,0.002660,5.491104,2.424374,0.000600
London,18.378665,0.923409,463.0,London,463.0,0.514563,10688.221680,0.505179,7.371926,2.165049,0.003747,7.044562,2.200647,0.001940,7.087800,2.576052,0.000524
North East,18.756243,0.974206,251.5,North East,251.5,0.466270,8281.648438,0.456207,5.246600,2.007937,0.000443,5.040420,2.125000,0.002369,3.750813,2.289683,0.000043
North West,18.888780,0.972397,633.5,North West,633.5,0.495268,9418.572266,0.455936,5.196530,1.832808,0.005411,5.044189,1.998423,0.006982,4.326658,2.303628,0.001881
Scotland,18.180234,0.969923,1745.0,Scotland,1745.0,0.486107,8709.897461,0.453888,5.418960,2.101690,0.001089,4.487819,1.983099,0.001750,3.831592,2.314237,0.000429
South East,18.342758,0.952381,734.5,South East,734.5,0.521769,10013.899414,0.463005,5.743897,1.767347,0.000971,6.004209,2.117687,0.002050,6.016914,2.534694,0.000174
South West,19.189720,0.958635,483.0,South West,483.0,0.490176,8671.809570,0.472033,5.408182,1.805584,0.003612,5.725100,2.091003,0.003493,5.459520,2.519131,0.000141
Wales,18.570539,0.959347,1500.0,Wales,1500.0,0.410863,8300.000000,0.468485,6.236510,2.132956,0.003785,5.326765,2.058980,0.003226,5.034431,2.377208,0.000773


In [34]:
output_df.reset_index(drop=True, inplace=True)
output_df

,std_dev_age,native_share,education_variability,region_name,job_variability,frac_employed,median_income,gini_index,Lindqvist_Ostling_S1,Abramowitz_Saunders_S1,Duca_Saving_S1,Lindqvist_Ostling_S2,Abramowitz_Saunders_S2,Duca_Saving_S2,Lindqvist_Ostling_S3,Abramowitz_Saunders_S3,Duca_Saving_S3
0,18.275323,0.958919,462.0,East Midlands,462.0,0.511351,8678.310547,0.449646,5.897372,1.818378,0.001149,6.021287,2.073514,0.004556,5.426529,2.431351,0.000332
1,18.773603,0.959739,459.0,East of England,459.0,0.503808,9591.965820,0.493658,5.312424,1.746464,0.001776,5.837405,2.103373,0.002660,5.491104,2.424374,0.000600
2,18.378665,0.923409,463.0,London,463.0,0.514563,10688.221680,0.505179,7.371926,2.165049,0.003747,7.044562,2.200647,0.001940,7.087800,2.576052,0.000524
3,18.756243,0.974206,251.5,North East,251.5,0.466270,8281.648438,0.456207,5.246600,2.007937,0.000443,5.040420,2.125000,0.002369,3.750813,2.289683,0.000043
4,18.888780,0.972397,633.5,North West,633.5,0.495268,9418.572266,0.455936,5.196530,1.832808,0.005411,5.044189,1.998423,0.006982,4.326658,2.303628,0.001881
5,18.180234,0.969923,1745.0,Scotland,1745.0,0.486107,8709.897461,0.453888,5.418960,2.101690,0.001089,4.487819,1.983099,0.001750,3.831592,2.314237,0.000429
6,18.342758,0.952381,734.5,South East,734.5,0.521769,10013.899414,0.463005,5.743897,1.767347,0.000971,6.004209,2.117687,0.002050,6.016914,2.534694,0.000174
7,19.189720,0.958635,483.0,South West,483.0,0.490176,8671.809570,0.472033,5.408182,1.805584,0.003612,5.725100,2.091003,0.003493,5.459520,2.519131,0.000141
8,18.570539,0.959347,1500.0,Wales,1500.0,0.410863,8300.000000,0.468485,6.236510,2.132956,0.003785,5.326765,2.058980,0.003226,5.034431,2.377208,0.000773
9,18.821393,0.971787,478.0,West Midlands,478.0,0.500522,8520.920898,0.446613,4.730679,1.735632,0.001685,4.876276,2.032393,0.001902,4.178338,2.412748,0.000548


In [35]:
output_df.to_csv(r"C:\Users\ramak\Desktop\Geog573_Lab\Geog_575_final_project\Final_output\polarization2000_data.csv")